In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns',None)

In [2]:
test = pd.read_csv("test.csv")
test = test.drop('id',axis=1)
train = pd.read_csv("train.csv")

In [3]:
X_1 = train.drop(['target','id'],axis=1)
y = train['target']

In [4]:
def transform(X):
    for n in ['nom_%d' % i for i in range(5,10)]:
        X[n] = X[n].apply(lambda x: int(x,16))
        X[n] -= X[n].mean()
        X[n] /= X[n].std()

    label = LabelEncoder()
    for col in ['bin_%d' % i for i in range(3,5)]:
        X[col] = label.fit_transform(X[col])

    dummy_nom = pd.get_dummies(X[['nom_%d' % i for i in range(5)]]).astype(np.int8)

    
    X = X.drop(['nom_%d' % i for i in range(5)],axis=1)X = pd.concat([X,pd.get_dummies(X[['nom_%d' % i for i in range(5)]]).astype(np.int8)],axis=1)

    columns=['day','month']
    for col in columns:
        X[col+'_sin'] = np.sin((2*np.pi*X[col])/max(X[col]))
        X[col+'_cos'] = np.cos((2*np.pi*X[col])/max(X[col]))
    X = X.drop(columns,axis=1)

    X['ord_3'] = X['ord_3'].apply(lambda x: ord(x)-97)
    X['ord_4'] = X['ord_4'].apply(lambda x: ord(x)-65)

    exp_dict = {'Grandmaster' : 4,
               'Master' : 3,
               'Expert': 2,
               'Contributor':1,
               'Novice': 0}
    X['ord_1'] = X['ord_1'].map(exp_dict)

    temp_dict = {'Lava Hot' : 5,
               'Boiling Hot' : 4,
               'Hot': 3,
                'Warm': 2,
               'Cold': 1,
               'Freezing': 0}
    X['ord_2'] = X['ord_2'].map(temp_dict)

    X['ord_5'] = X['ord_5'].apply(lambda x: int(x,36))
    X['ord_5'] -= X['ord_5'].mean()
    X['ord_5'] /= X['ord_5'].std()
    return(X)

In [5]:
X_1 = transform(X_1)

X_1.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,nom_0_Blue,nom_0_Green,nom_0_Red,nom_1_Circle,nom_1_Polygon,nom_1_Square,nom_1_Star,nom_1_Trapezoid,nom_1_Triangle,nom_2_Axolotl,nom_2_Cat,nom_2_Dog,nom_2_Hamster,nom_2_Lion,nom_2_Snake,nom_3_Canada,nom_3_China,nom_3_Costa Rica,nom_3_Finland,nom_3_India,nom_3_Russia,nom_4_Bassoon,nom_4_Oboe,nom_4_Piano,nom_4_Theremin,day_sin,day_cos,month_sin,month_cos
0,0,0,0,1,1,-0.569917,-0.955851,-0.359670,0.909181,-1.091591,2,4,1,7,3,-0.182033,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,9.749279e-01,-0.222521,0.866025,0.500000
1,0,1,0,1,1,0.745176,1.608589,-0.966682,-0.694388,1.624983,1,4,3,0,0,-1.432595,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,-2.449294e-16,1.000000,-0.866025,-0.500000
2,0,0,0,0,1,-0.976732,-1.615043,0.462505,1.275011,0.626839,1,2,5,7,17,-0.371851,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,-2.449294e-16,1.000000,0.866025,0.500000
3,0,1,0,0,1,1.563110,-0.662449,1.392626,-0.721130,0.032449,1,4,4,8,3,-0.163424,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,9.749279e-01,-0.222521,0.500000,0.866025
4,0,0,0,0,0,-0.435189,-1.312890,-1.694052,1.013609,0.667224,1,4,0,0,17,0.614456,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,-2.449294e-16,1.000000,-0.866025,-0.500000


In [6]:
X_train,X_test,y_train,y_test=train_test_split(X_1,y,test_size=0.2)

In [7]:
# logi = LogisticRegression()
# logi.fit(X_train,y_train)

# y_pre = logi.predict(X_test)
# print(accuracy_score(y_test,y_pre))

# preds = logi.predict_proba(test)
# preds_prob = [preds[i][1] for i in range(len(preds))]
# outp = pd.DataFrame(preds_prob,columns=['target'])
# outp['target'] = preds_prob
# outp.index = range(300000,500000)

In [8]:
data_dm = xgb.DMatrix(data=X_1,label=y)

/home/stephen/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/stephen/.local/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [9]:
test = transform(test)

In [10]:
xg_reg = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [11]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict_proba(test)

[13:50:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [12]:
 params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                 'max_depth': 5, 'alpha': 10}

# cv_results = xgb.cv(dtrain=data_dm, params=params, nfold=3,
#                     num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [35]:
preds_prob = [preds[i][1] for i in range(len(preds))]
outp = pd.DataFrame(preds_prob,columns=['target'])
outp['target'] = preds_prob
outp.index = range(300000,500000)

In [37]:
outp.to_csv("submissions.csv")